In [4]:
'''

    Script to process dataset, calculating TextBlob and VADER polarity and subjectivity scores

    Assumes dataset features contains columns titled "text" and "title" 

    Assumes input and output files are CSV

    Assumes that class indicator column is called 'label'
    TODO: If this needs to be a program that the prof/TA can use, then CLI args should be used for file names

'''


import os
import pandas as pd
from textblob import *
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *




read_file = r"../data/Des_fake_news/LIAR.csv"
read_sheet = "valid"
write_file = r"../data/Des_fake_news/LIAR_PROCESSED_2.csv"

#data = pd.read_excel(read_file, read_sheet)
data = pd.read_csv(read_file)

data = data.dropna(axis=0)
data

,id,class,text,title,speaker,job title,state,party,barelytrues,falses,halftrues,mostlystrues,pantsonfires,context,flag
0,11972.json,TRUE,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,1
1,11685.json,FALSE,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,0
2,11096.json,FALSE,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,0
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,0
4,7070.json,TRUE,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0,0,0,0,0,in a post on Facebook.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8463,7013.json,barely-true,Says U.S. Rep. Charles Bass wants to privatize...,social-security,ann-mclane-kuster,Attorney,New Hampshire,democrat,2,1,3,0,0,"an ad, Ã¢â‚¬Å“Janice,Ã¢â‚¬Â released Septembe...",0
8464,2661.json,pants-fire,"In the past two years, Democrats have spent mo...","federal-budget,history",eric-cantor,House Majority Leader,Virginia,republican,9,6,4,4,4,an interview on Comedy Central's Daily Show wi...,0
8465,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference,0
8466,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,69,76,7,a speech on the economy,1


In [5]:
'''
    Natural language preprocessing

    Remove punctuation, make all words lowercase, and lemmatize
'''

from nltk.corpus import wordnet
from textblob import Word
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string

'''
    NLTK has a model to tag words as adjectives, nouns, etc,
    but NLTK uses wordnet for lemmatization. wordnet only uses
    four possible tags, while NLTK returns tons of unique ones

    This function transforms NLTK tags to wordnet tags for lemmatization
'''
def nltk_tag_to_wordnet(tag: str) -> str:
    if tag[0] == "J":
        return wordnet.ADJ
    elif tag[0] == "V":
        return wordnet.VERB
    elif tag[0] == "N":
        return wordnet.NOUN
    elif tag[0] == "R":
        return wordnet.ADV
    else:
        return ""
    
'''
    Remove non-alphabetical characters and punctuation
'''
def keep_only_alphabetic(s: str) -> str:
    temp = "".join([" " if i in string.punctuation else i for i in s])
    return "".join([i for i in temp if (ord(i) <= 90 and ord(i) >= 65) or (ord(i) <= 122 and ord(i) >= 97) or i.isspace()])

'''
    Take a string of text, tokenize it, and return a list of lemmatized tokens
'''
def lemmatize_words(s: str) -> list[str]:
    lemmer = nltk.stem.WordNetLemmatizer()
    words = [i.lower() for i in word_tokenize(s)]       # tokenize and lowercase
    words = [i for i in words if i not in stopwords.words("english")]   # remove stopwords
    words = list(filter(lambda x: nltk_tag_to_wordnet(x[1]) !="", pos_tag(words)))  # remove invalid lemmatization words and tags
    words = [lemmer.lemmatize(i[0], nltk_tag_to_wordnet(i[1])) for i in words]  #  lemmatize words
    return words

'''
    Combine all functions above to pre-process strng
'''
def pre_process_text(text: str) -> str:
    s = keep_only_alphabetic(text)
    lemmatized = lemmatize_words(s)
    return " ".join(lemmatized)






In [3]:
data["text"] = data.apply(lambda x: pre_process_text(x["text"]), axis=1)
data["title"] = data.apply(lambda x: pre_process_text(x["title"]), axis=1)

KeyboardInterrupt: 

In [ ]:
data

In [ ]:
# SKIP FOR WELFAKE

'''
def to_class(s):
    if s == "real":
        return 1
    return 0
'''


def liar_class_process(s: str) -> int:
    if s.lower() == "true":
        return 1
    elif s.lower() == "mostly-true":
        return 1
    else:
        return 0


In [ ]:
#data["flag"] = data.apply(lambda x: to_class(x["flag"]), axis=1)
#data

if "liar" in read_file.lower():
    data["class"] = data.apply(lambda x: liar_class_process(x["class"]), axis=1)

data

In [ ]:
data["text_tb_pol"] = data.apply(lambda x: TextBlob(x["text"]).polarity, axis=1)
data["text_tb_sub"] = data.apply(lambda x: TextBlob(x["text"]).subjectivity, axis=1)

data["title_tb_pol"] = data.apply(lambda x: TextBlob(x["title"]).polarity, axis=1)
data["title_tb_sub"] = data.apply(lambda x: TextBlob(x["title"]).subjectivity, axis=1)
data

In [ ]:
analyzer = SentimentIntensityAnalyzer()

data["title_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["title"]), axis=1)
data["title_vader_comp"] = data.apply(lambda x: x["title_vader_scores"]["compound"], axis=1)
data["title_vader_neg"] = data.apply(lambda x: x["title_vader_scores"]["neg"], axis=1)
data["title_vader_neu"] = data.apply(lambda x: x["title_vader_scores"]["neu"], axis=1)
data["title_vader_pos"] = data.apply(lambda x: x["title_vader_scores"]["pos"], axis=1)
data = data.drop(["title_vader_scores"], axis=1)

data["text_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["text"]), axis=1)
data["text_vader_comp"] = data.apply(lambda x: x["text_vader_scores"]["compound"], axis=1)
data["text_vader_neg"] = data.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
data["text_vader_neu"] = data.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
data["text_vader_pos"] = data.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
data = data.drop(["text_vader_scores"], axis=1)

In [ ]:
def discrete_vader(neg, neu, pos):
    return np.argmax([neg, neu, pos])
    
def discrete_textblob(pol):
    if pol <= -0.33:
        return 0
    elif pol > -0.33 and pol < 0.33:
        return 1
    else:
        return 2

def discrete_textblob_sub(pol):
    if pol < 0.5:
        return 0
    return 1

In [ ]:

data["title_vader_class"] = data.apply(lambda x: discrete_vader(x["title_vader_neg"], x["title_vader_neu"], x["title_vader_pos"]), axis=1)
data["text_vader_class"] = data.apply(lambda x: discrete_vader(x["text_vader_neg"], x["text_vader_neu"], x["text_vader_pos"]), axis=1)

data["text_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["text_tb_pol"]), axis=1)
data["text_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["text_tb_sub"]), axis=1)

data["title_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["title_tb_pol"]), axis=1)
data["title_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["title_tb_sub"]), axis=1)
data


In [ ]:
#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

In [ ]:
'''
    Begin Neural Net for news text
'''

#data = pd.read_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv")
'''text only'''
features = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment = np.argmax(imdb_model.predict(features), axis=-1)

tweets_model = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment = np.argmax(tweets_model.predict(features), axis=-1)

data["text_NN_imdb"] = imdb_sentiment
data["text_NN_tweets"] = tweets_sentiment

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

In [ ]:
'''
    Begin Neural Net for news title
'''


'''text only'''
features = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model_two = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment_two = np.argmax(imdb_model_two.predict(features), axis=-1)

tweets_model_two = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment_two = np.argmax(tweets_model_two.predict(features), axis=-1)

data["title_NN_imdb"] = imdb_sentiment_two
data["title_NN_tweets"] = tweets_sentiment_two

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

In [ ]:
'''
    Begin log regression for text
'''
from sklearn.linear_model import LogisticRegression
import pickle

texts = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["text_log_imdb"] = imdb_text_log.predict(texts)
data["text_log_tweets"] = imdb_text_log.predict(texts)





In [ ]:
'''
    Begin log regression for title
'''
from sklearn.linear_model import LogisticRegression
import pickle

titles = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["title_log_imdb"] = imdb_text_log.predict(texts)
data["title_log_tweets"] = imdb_text_log.predict(texts)


data.to_csv(write_file, index=False)

In [ ]:
data.to_csv(write_file)